In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np



In [2]:
import json

with open('../project-data/train-claims.json', 'r') as tclaim_file:
    train_claim_data = json.load(tclaim_file)

# Read in development data (claim)
with open('../project-data/dev-claims.json', 'r') as dclaim_file:
    dev_claim_data = json.load(dclaim_file)

# Read in test data (claim)
with open('../project-data/test-claims-unlabelled.json', 'r') as uclaim_file:
    test_claim_data = json.load(uclaim_file)

# Read in evidence data
with open('../project-data/evidence.json', 'r') as evi_file:
    evi_data = json.load(evi_file)
    
claim = train_claim_data['claim-1937']['claim_text']
evidence_pos = evi_data[train_claim_data['claim-1937']['evidences'][0]]
evidence_neg = evi_data["evidence-338219"]
print(claim.lower())
print(evidence_pos)
print(evidence_neg)




not only is there no scientific evidence that co2 is a pollutant, higher co2 concentrations actually help ecosystems support more plant and animal life.
At very high concentrations (100 times atmospheric concentration, or greater), carbon dioxide can be toxic to animal life, so raising the concentration to 10,000 ppm (1%) or higher for several hours will eliminate pests such as whiteflies and spider mites in a greenhouse.
While ‘climate change’ can be due to natural forces or human activity, there is now substantial evidence to indicate that human activity – and specifically increased greenhouse gas (GHGs) emissions – is a key factor in the pace and extent of global temperature increases.


In [3]:
#提取train用到了的evidence

train_evi_id = []
train_claim_id = []
train_claim_text = []
for claim_id,claim_value in train_claim_data.items():
    train_evi_id=train_evi_id+claim_value['evidences']
    train_claim_id.append(claim_id)
    train_claim_text.append(claim_value['claim_text'].lower())
    

#去除重复
train_evi_id = list(set(train_evi_id))
print(len(train_evi_id))

train_evi_text = []

for id in train_evi_id:
    train_evi_text.append(evi_data[id].lower())
print(len(train_evi_text))

3121
3121


In [4]:
full_evidence_id = [ k for k,v in evi_data.items()]
full_evidence_text  = [ v for k,v in evi_data.items()]

print(full_evidence_id[1])

evidence-1


In [5]:
import pandas as pd
import sklearn as sk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer(stop_words="english", use_idf=True)

# tfidf_vec = tfidf_vectorizer.fit(train_claim_text+train_evi_text)
tfidf_vec = tfidf_vectorizer.fit(train_claim_text+full_evidence_text)


train_claim_emb_list = tfidf_vec.transform(train_claim_text)
# train_evi_emb_list = tfidf_vec.transform(train_evi_text)

full_evi_emb_list = tfidf_vec.transform(full_evidence_text)



In [14]:
full_evi_emb_list.shape

(1208827, 605251)

In [22]:
# print(cosine_similarity(tfidf_vec.transform([claim]), tfidf_vec.transform(train_evi_text)))

In [11]:
# test_claim_text = []
k = 2


# with open('../project-data/test-claims-unlabelled.json', 'r') as dclaim_file:
#     test_out_temp = json.load(dclaim_file)

for claim_id,claim_value in test_claim_data.items():
    # train_evi_id=train_evi_id+claim_value['evidences']
    # train_claim_id.append(claim_id)
    test_claim_text = claim_value['claim_text']
    test_claim_emb = tfidf_vec.transform([test_claim_text])
    evi_sim_dict = {}
    sim = cosine_similarity(test_claim_emb, full_evi_emb_list)[0]
    
    for i in range(len(sim)):
        evi_sim_dict[full_evidence_id[i]] = sim[i]
    
    s_sim = [(k, v) for k, v in sorted(evi_sim_dict.items(), key=lambda item: item[1],reverse=True)][:k]
    # print(s_sim)
    sel_sim = [k for k,v in s_sim]
    # print(sel_sim)
    test_claim_data[claim_id]["evidences"] = sel_sim
    test_claim_data[claim_id]["claim_label"] = "SUPPORTS"
    
    # test_claim_data['claim_label']


print(test_claim_data)
# for i in range(len(dev_claim_id)):
#     claim_embedding = get_embedding(dev_claim_text[i])[0]
#     best3 = get_best_3_evi(claim_embedding)
#     e_list= [id for (id, score) in best3]
#     out_temp_dev[dev_claim_id[i]]["evidences"] = e_list

json_object = json.dumps(test_out_temp, indent=4)
 
# Writing to sample.json
# with open("../project-data/dev_predict.json", "w") as outfile:
#     outfile.write(json_object)

{'claim-2967': {'claim_text': 'The contribution of waste heat to the global climate is 0.028 W/m2.', 'evidences': ['evidence-308923', 'evidence-188763'], 'claim_label': 'SUPPORTS'}, 'claim-979': {'claim_text': '“Warm weather worsened the most recent five-year drought, which included the driest four-year period on record in terms of statewide precipitation.', 'evidences': ['evidence-812998', 'evidence-937309'], 'claim_label': 'SUPPORTS'}, 'claim-1609': {'claim_text': 'Greenland has only lost a tiny fraction of its ice mass.', 'evidences': ['evidence-962481', 'evidence-1200633'], 'claim_label': 'SUPPORTS'}, 'claim-1020': {'claim_text': '“The global reef crisis does not necessarily mean extinction for coral species.', 'evidences': ['evidence-161852', 'evidence-394252'], 'claim_label': 'SUPPORTS'}, 'claim-2599': {'claim_text': 'Small amounts of very active substances can cause large effects.', 'evidences': ['evidence-834109', 'evidence-432221'], 'claim_label': 'SUPPORTS'}, 'claim-2110': {'

In [13]:
file_path = '../test-claims-predictions.json'
with open(file_path, 'w') as json_file:
    json.dump(test_claim_data, json_file)